In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, BooleanType, LongType
from delta.tables import DeltaTable

CATALOGO_ORIGEM = "spotify_analytics"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "tb_bronze_search"

CATALOGO_DESTINO = "spotify_analytics"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_tracks"
TABELA_INVALIDOS_DESTINO = "tb_tracks_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

### Configuração

Define origem (Bronze), destino (Silver) e tabela de auditoria (inválidos).

### Schema Explícito para JSON

Define estrutura esperada do JSON da API Spotify Search:
- `tracks.items[]`: Array de tracks
- Cada track tem: id, name, album, popularity, duration_ms, explicit
- Album contém: name, album_type, release_date

In [ ]:
# Schema do JSON customizado (array de tracks direto na raiz)
spotify_schema = StructType([
    StructField("items", ArrayType(StructType([
        StructField("id", StringType(), True),
        StructField("name", StringType(), True),
        StructField("popularity", IntegerType(), True),
        StructField("duration_ms", LongType(), True),
        StructField("explicit", BooleanType(), True),
        StructField("album", StructType([
            StructField("name", StringType(), True),
            StructField("album_type", StringType(), True),
            StructField("release_date", StringType(), True)
        ]), True)
    ])), True)
])

### UDF: Normalização de Data de Lançamento

A API retorna datas em formatos variáveis:
- "2024" → normaliza para "2024-01-01"
- "2024-05" → normaliza para "2024-05-01"
- "2024-05-15" → mantém

Permite conversão para DATE de forma consistente.

In [ ]:
@F.udf(returnType=StringType())
def normalize_release_date(date_string):
    """Normaliza formatos variáveis de data para yyyy-MM-dd"""
    if date_string is None:
        return None
    
    parts = date_string.split("-")
    
    if len(parts) == 1:  # Apenas ano
        return f"{parts[0]}-01-01"
    elif len(parts) == 2:  # Ano-mês
        return f"{parts[0]}-{parts[1]}-01"
    else:  # Data completa
        return date_string

### Leitura Incremental do Bronze

Lê apenas a última carga (max ingestion_date):
- Parseia JSON usando schema explícito
- Explode array de tracks
- Normaliza data de lançamento
- Remove duplicatas por track_id

In [ ]:
# Lê última carga do Bronze
df_bronze = spark.read.table(nome_tabela_origem)

max_dt_ingestao = (
    df_bronze
    .agg(F.max(F.col("ingestion_date")).alias("max_ts"))
    .first()["max_ts"]
)

# Parseia JSON e explode tracks
df_parsed = (
    df_bronze
    .filter(F.col("ingestion_date") == F.lit(max_dt_ingestao))
    .withColumn("parsed_data", F.from_json(F.col("raw_json"), spotify_schema))
    .select(
        F.explode(F.col("parsed_data.items")).alias("track"),
        "ingestion_date",
        "source_file"
    )
)

# Extrai e limpa campos
df_limpo = (
    df_parsed
    .select(
        F.col("track.id").alias("track_id"),
        F.trim(F.col("track.name")).alias("track_name"),
        F.trim(F.col("track.album.name")).alias("album_name"),
        F.col("track.album.album_type").alias("album_type"),
        F.to_date(
            normalize_release_date(F.col("track.album.release_date")), 
            "yyyy-MM-dd"
        ).alias("release_date"),
        F.col("track.popularity").alias("popularity"),
        F.col("track.duration_ms").alias("duration_ms"),
        F.col("track.explicit").alias("explicit"),
        F.col("ingestion_date").alias("dt_ingestion"),
        F.lit("spotify_api_search").alias("dc_origem")
    )
    .withColumn("release_year", F.year(F.col("release_date")))
    .dropDuplicates(["track_id"])
)

print(f"Registros lidos e limpos: {df_limpo.count()}")

### Validação de Qualidade

Adiciona flags de validação:
- **flag_id_valido**: track_id IS NOT NULL
- **flag_nome_valido**: track_name IS NOT NULL AND length > 1
- **flag_duracao_valida**: duration_ms > 0
- **flag_qualidade**: "OK" se todas flags TRUE, senão "ERRO"

Split em df_validos e df_invalidos.

In [ ]:
df_validacao = (
    df_limpo
    .withColumn("flag_id_valido", F.col("track_id").isNotNull())
    .withColumn("flag_nome_valido", 
        F.col("track_name").isNotNull() & (F.length(F.col("track_name")) > 1)
    )
    .withColumn("flag_duracao_valida", 
        F.col("duration_ms").isNotNull() & (F.col("duration_ms") > 0)
    )
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_id_valido") &
            F.col("flag_nome_valido") &
            F.col("flag_duracao_valida"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

# Remove flags dos registros válidos
df_silver = df_validos.select(
    "track_id", "track_name", "album_name", "album_type",
    "release_date", "release_year", "popularity", 
    "duration_ms", "explicit", "dt_ingestion", "dc_origem"
)

print(f"Registros válidos: {df_validos.count()}")
print(f"Registros inválidos: {df_invalidos.count()}")

### MERGE de Registros Válidos

Faz MERGE (UPSERT) em tb_tracks:
- **MATCHED**: Atualiza registro existente
- **NOT MATCHED**: Insere novo registro

Operação idempotente (pode ser re-executada sem gerar duplicatas).

In [ ]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.track_id = origem.track_id"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")

### OVERWRITE de Registros Inválidos

Sobrescreve tb_tracks_invalidos com registros rejeitados da última carga.
Contém flags de validação para rastreabilidade de problemas.

In [ ]:
df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria")

### Verificação Final

Exibe métricas da tabela Silver.

In [ ]:
# Total de tracks
total = spark.table(nome_tabela_destino).count()
print(f"Total de tracks na Silver: {total}")

# Top 5 tracks mais populares
spark.sql(f"""
    SELECT track_name, album_name, popularity
    FROM {nome_tabela_destino}
    ORDER BY popularity DESC
    LIMIT 5
""").show(truncate=False)